In [1]:
import pandas as pd
import json
from get_wiki_data import WikiDataLoader

COMPMIX_IR_PATH = "CompMix-IR/kb_wikipedia_mixed_rd.json"
COMPMIX_PATH = "CompMix/train_set.jsonl"

In [7]:
dataloader = WikiDataLoader()
infoboxes, wikitables = dataloader.load_tables(qid="Q54935007")

display(len(infoboxes))
display(len(wikitables))

0

25

In [15]:
def read_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        qa = f.readlines()
    qa = [json.loads(line) for line in qa]
    return pd.DataFrame(qa)


df = read_jsonl(COMPMIX_PATH)
df.head()



,question_id,question,domain,entities,answers,answer_src,answer_text,convmix_question_id
0,3642,What is the genre of the tv series High Seas?,tvseries,"[{'id': 'Q186424', 'label': 'detective fiction'}]","[{'id': 'Q59591953', 'label': 'High Seas'}]",kb,detective fiction,1236-0
1,2082,What year was M.O.V.E first formed?,music,"[{'id': 'Q285547', 'label': 'Move'}]","[{'id': '1997-01-01T00:00:00Z', 'label': '1997'}]",infobox,1997,694-1
2,4693,What year was the inception of the soccer club...,soccer,"[{'id': 'Q18656', 'label': 'Manchester United ...","[{'id': '1878-01-01T00:00:00Z', 'label': '1878'}]",kb,1878,1588-4
3,2149,Who were the creators of the situation comedy ...,tvseries,"[{'id': 'Q615944', 'label': 'The Mary Tyler Mo...","[{'id': 'Q95125', 'label': 'James L. Brooks'},...",kb,James L. Brooks and Allan Burns,716-2
4,2212,Which country did the TV series Coupling origi...,tvseries,"[{'id': 'Q1137587', 'label': 'Coupling'}]","[{'id': 'Q145', 'label': 'United Kingdom'}]",infobox,United Kingdom,737-0


In [17]:

df_table = df[df['answer_src']=='table']
df_table.head()


,question_id,question,domain,entities,answers,answer_src,answer_text,convmix_question_id
11,4052,What is Russell Crowe's date of birth?,movies,"[{'id': 'Q129817', 'label': 'Russell Crowe'}]","[{'id': '1964-04-07T00:00:00Z', 'label': '7 Ap...",table,7 April 1964,1372-3
14,1057,Who won the Best Actor award in the Los Angele...,movies,"[{'id': 'Q47221', 'label': 'Taxi Driver'}]","[{'id': 'Q36949', 'label': 'Robert De Niro'}]",table,Robert De Niro,352-2
31,1575,Lin-Manuel Miranda played the role of who in t...,music,"[{'id': 'Q28659892', 'label': 'list of awards ...","[{'id': 'Q178903', 'label': 'Alexander Hamilto...",table,Alexander Hamilton,524-4
53,2992,which date was Emily in Paris season 1 released?,tvseries,"[{'id': 'Q64584978', 'label': 'Emily in Paris'}]","[{'id': '2020-10-02T00:00:00Z', 'label': '2 Oc...",table,"October 2, 2020",1017-0
54,4249,Winner of the 2021 MLS cup?,soccer,"[{'id': 'Q577698', 'label': 'MLS Cup'}]","[{'id': 'Q13361030', 'label': 'New York City F...",table,New York City FC,1439-0


## 下载table数据


In [19]:
table_path = "CompMix/tableqa_sample.csv"
df = pd.read_csv(table_path)
df.head()



,question_id,question,domain,entities,answers,answer_src,answer_text,convmix_question_id
0,4052,What is Russell Crowe's date of birth?,movies,"[{'id': 'Q129817', 'label': 'Russell Crowe'}]","[{'id': '1964-04-07T00:00:00Z', 'label': '7 Ap...",table,7 April 1964,1372-3
1,1057,Who won the Best Actor award in the Los Angele...,movies,"[{'id': 'Q47221', 'label': 'Taxi Driver'}]","[{'id': 'Q36949', 'label': 'Robert De Niro'}]",table,Robert De Niro,352-2
2,1575,Lin-Manuel Miranda played the role of who in t...,music,"[{'id': 'Q28659892', 'label': 'list of awards ...","[{'id': 'Q178903', 'label': 'Alexander Hamilto...",table,Alexander Hamilton,524-4
3,2992,which date was Emily in Paris season 1 released?,tvseries,"[{'id': 'Q64584978', 'label': 'Emily in Paris'}]","[{'id': '2020-10-02T00:00:00Z', 'label': '2 Oc...",table,"October 2, 2020",1017-0
4,4249,Winner of the 2021 MLS cup?,soccer,"[{'id': 'Q577698', 'label': 'MLS Cup'}]","[{'id': 'Q13361030', 'label': 'New York City F...",table,New York City FC,1439-0


In [37]:
def get_entities(df):
    entities = []
    for _,row in df[0:10].iterrows():
        e = row['entities'].replace("'", '"')
        ej = json.loads(e)
        for entity in ej:
            if entity not in entities:
                entities.append(entity)
    return entities

entities = get_entities(df)
entities[:5]

[{'id': 'Q129817', 'label': 'Russell Crowe'},
 {'id': 'Q47221', 'label': 'Taxi Driver'},
 {'id': 'Q28659892',
  'label': 'list of awards and nominations received by Lin-Manuel Miranda'},
 {'id': 'Q19320959', 'label': 'Hamilton'},
 {'id': 'Q64584978', 'label': 'Emily in Paris'}]

In [ ]:
for entity in entities:
    qid = entity['id']
    infoboxes, wikitables = dataloader.load_tables(qid)

In [6]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/University_of_California,_Berkeley'
infoboxes = read_html(page, index_col=0, attrs={"class":"infobox"})
wikitables = read_html(page, index_col=0, attrs={"class":"wikitable"})

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?